In [1]:
# DO you filling missing values before spliting the dataset into train and test or after
# DO you convert to numeric before spliting the dataset into train and test or after
#enter the date variable correctly
#convert others to female later in gender
#decide on date_first_booking, but ight replace with -9999 or leave it like that 
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from datetime import datetime

In [3]:
train = pd.read_csv('train_users_2.csv', parse_dates=['date_account_created','timestamp_first_active','date_first_booking'])
train.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,NaT,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,2009-05-23 17:48:09,NaT,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,2009-06-09 23:12:47,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,2009-10-31 06:01:29,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,2009-12-08 06:11:05,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [4]:
train['timestamp_first_active_to_date'] = pd.to_datetime(train['timestamp_first_active'], format='%Y%m%d%H%M%S')


In [5]:
train['timestamp_first_active_to_date'] = train['timestamp_first_active_to_date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [6]:
#impute age missing values with the median

train['age'].fillna(train['age'].median() , inplace = True)

In [7]:
#impute first_affiliate_tracked missing values with the mode

train['first_affiliate_tracked'].fillna(train['first_affiliate_tracked'].mode()[0] , inplace = True)

In [8]:
#Convert to numeric
labelencoder_X  = LabelEncoder()

train['gender'] = labelencoder_X.fit_transform(train['gender'])

In [9]:

train['language'] = labelencoder_X.fit_transform(train['language'])

In [10]:
train['affiliate_channel'] = labelencoder_X.fit_transform(train['affiliate_channel'])

In [11]:
train['affiliate_provider'] = labelencoder_X.fit_transform(train['affiliate_provider'])

In [12]:
train['first_affiliate_tracked'] = labelencoder_X.fit_transform(train['first_affiliate_tracked'])

In [13]:
train['signup_app'] = labelencoder_X.fit_transform(train['signup_app'])

In [14]:
train['first_device_type'] = labelencoder_X.fit_transform(train['first_device_type'])

In [15]:
train['first_browser'] = labelencoder_X.fit_transform(train['first_browser'])

In [16]:
train['country_destination'] = labelencoder_X.fit_transform(train['country_destination'])

In [17]:
train['signup_method'] = labelencoder_X.fit_transform(train['signup_method'])

In [18]:
#was a first booking made
train['first_booking_made'] = pd.notnull(train.loc[:, 'date_first_booking'])

In [19]:
#convert to integer 
train['first_booking_made'] = train['first_booking_made'].astype(int)

In [20]:
train['timestamp_first_active'] = train['timestamp_first_active'].astype(int)

### FEATURE ENG.

In [21]:
# date_account_created  timestamp_first_active  date_first_booking

In [22]:
#convert type to date type
train['timestamp_first_active_to_date'] = pd.to_datetime(train['timestamp_first_active_to_date'])

In [23]:
train['number_of_days_btw_first_active_first_booking'] = train['date_first_booking'] - train['timestamp_first_active_to_date']

In [24]:
train['month_of_first_active'] = train['timestamp_first_active_to_date'].apply(lambda x: x.strftime('%m'))

In [25]:
train['month_account_created'] = train['date_account_created'].apply(lambda x: x.strftime('%m'))

In [26]:
train['month_of_first_booking'] =  train['date_first_booking'].map(lambda x: x.strftime('%m') if pd.notnull(x) else '')

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
# train_without_dates = train[['gender', 'age', 'signup_method','signup_flow','language','affiliate_channel','affiliate_provider','first_affiliate_tracked','signup_app','first_device_type','first_browser','first_booking_made','month_of_first_active','month_account_created','month_of_first_booking','number_of_days_btw_first_active_first_booking', 'country_destination']]
train_without_dates = train[['gender', 'age', 'signup_method','signup_flow','language','affiliate_channel','affiliate_provider','first_affiliate_tracked','signup_app','first_device_type','first_browser','first_booking_made','number_of_days_btw_first_active_first_booking', 'country_destination']]

In [29]:
train_without_dates['delta_join_booking'] = train_without_dates['number_of_days_btw_first_active_first_booking'].apply(lambda x: x.days)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
y = train_without_dates['country_destination']
X = train_without_dates.drop(['country_destination'], axis=1)

In [31]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [32]:
X_train = X_train.drop(['number_of_days_btw_first_active_first_booking','delta_join_booking'], axis=1)
X_test = X_test.drop(['number_of_days_btw_first_active_first_booking','delta_join_booking'], axis=1)

In [33]:
X = X.drop(['number_of_days_btw_first_active_first_booking','delta_join_booking'], axis=1)

In [34]:
X['target'] = y

In [35]:
X.to_csv("groomed_train_users.csv")

In [36]:
from sklearn.linear_model import LogisticRegression

In [37]:
lr = LogisticRegression()

In [38]:
fitted = lr.fit(X_train, y_train)

In [39]:
fitted.score(X_test, y_test)

0.8747852752026576

In [40]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(fitted, X, y, cv=5)

In [41]:
scores

array([0.94760511, 0.94776661, 0.94581996, 0.94651893, 0.94269516])

In [42]:
train.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,...,signup_app,first_device_type,first_browser,country_destination,timestamp_first_active_to_date,first_booking_made,number_of_days_btw_first_active_first_booking,month_of_first_active,month_account_created,month_of_first_booking
0,gxn3p5htnn,2010-06-28,1237437175000000000,NaT,0,34.0,1,0,5,2,...,2,3,8,7,2009-03-19,0,NaT,03,06,
1,820tgsjxq7,2011-05-25,1243100889000000000,NaT,2,38.0,1,0,5,7,...,2,3,8,7,2009-05-23,0,NaT,05,05,
2,4ft3gnwmtx,2010-09-28,1244589167000000000,2010-08-02,1,56.0,0,3,5,2,...,2,6,21,10,2009-06-09,1,419 days,06,09,08
3,bjjt8pjhuk,2011-12-05,1256968889000000000,2012-09-08,1,42.0,1,0,5,2,...,2,3,17,11,2009-10-31,1,1043 days,10,12,09
4,87mebub9p4,2010-09-14,1260252665000000000,2010-02-18,0,41.0,0,0,5,2,...,2,3,8,10,2009-12-08,1,72 days,12,09,02


In [43]:
%notify

<IPython.core.display.Javascript object>